---

## Задание 1: Теоретическое описание задачи

Задача извлечения именованных сущностей (Named Entity Recognition, NER) — это ключевая подзадача обработки естественного языка (NLP), направленная на автоматическое обнаружение и классификацию сущностей в неструктурированном тексте. К типичным категориям сущностей относятся:

- `PER` — персоны (люди),
- `ORG` — организации,
- `LOC` — географические объекты,
- `EVT` — события,
- `PRO` — продукты или объекты.

**Классические методы:**
До распространения нейронных сетей основными подходами были:

- Правила и шаблоны (rule-based systems),
- Статистические модели: HMM (Hidden Markov Models), CRF (Conditional Random Fields),
- Feature engineering — ручное составление признаков на основе лингвистических свойств.

**Современные подходы (LLM):** 

С появлением больших языковых моделей (LLM) стало возможным извлечение сущностей в zero-shot или few-shot режимах, без дополнительного обучения. GigaChat, как представитель LLM, способен воспринимать структурированный prompt и возвращать списки сущностей, опираясь на контекст.

**Метрики качества:**

Для оценки качества предсказания применяются:

Precision (точность),
Recall (полнота),
F1-score — гармоническое среднее между precision и recall.
Оценка может проводиться:

По каждой сущности (class-wise),
В среднем (macro/micro-average).

---

##  Задание 2: Загрузка и подготовка датасета

**Формулировка:**  
Реализуйте чтение датасета в pandas DataFrame с обязательными колонками:
- `document_id`
- `document_text`
- `entity`
- `gold_answer`

Выведите шапку датафрейма.


In [ ]:
# Загрузка и отображение DataFrame

---

##  Задание 3: Формирование prompt-функции для LLM

**Формулировка:**  
Напишите функцию, которая принимает на вход строку датафрейма и выдает текст входного сообщения для LLM




In [ ]:
# Код для задания 3

---

##  Задание 4: Ручное получение ответов от GigaChat и сохранение

**Формулировка:**  
Получите ответы GigaChat для всех документов. Документов всего 9, поэтому сделать это можно вручную, пользуясь веб-интерфейсом GigaChat или ботом в ВК или Телеграме. 




In [ ]:
# Код для задания 4

---

##  Задание 5: Реализация метрики score_fn() и юнит-тесты

**Формулировка:**  
<сюда можно вставить формулировку из ТЗ>




In [ ]:
# Код для задания 5

---

##  Задание 6: Расчёт метрик и визуализация по сущностям/документам

**Формулировка:**  
<сюда можно вставить формулировку из ТЗ>



In [ ]:
# Код для задания 6


---

##  Задание 7: Анализ зависимости качества от длины документа

**Формулировка:**  
<сюда можно вставить формулировку из ТЗ>


In [ ]:
# Код для задания 7

---

##  Задание 8: Анализ ошибок, слабых и сильных сторон модели

**Формулировка:**  




In [ ]:
# Код для задания 8

---

##  Задание 9: Выводы по проекту и самооценка

**Формулировка:**  





In [ ]:
# Код для задания 9